In [1]:
from streaming import StreamingDataset

/Users/zackankner/Desktop/ai/llm-foundry/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
domains = {
    'c4': 'oci://mosaicml-internal-dataset-c4/sem-dedupe/23-04-05/dataset/pretokencat/EleutherAI-gpt-neox-20b/0pt8/',
    'markdown': 'oci://mosaicml-internal-datasets/stack-split-neox/markdown/',
    'mc4': 'oci://mosaicml-internal-dataset-mc4/filtered/en/filter2-gptneox-tok/',
    'redpajama': 'oci://mosaicml-internal-dataset-red-pajama/cc/gptneox-tok/',
    'redpajama-arxiv': 'oci://mosaicml-internal-dataset-red-pajama/arxiv/gptneox-tok/',
    'redpajama-books': 'oci://mosaicml-internal-dataset-red-pajama/books/gptneox-tok/',
    'redpajama-stackexchange': 'oci://mosaicml-internal-dataset-red-pajama/stackexchange/gptneox-tok/',
    'redpajama-wiki': 'oci://mosaicml-internal-dataset-red-pajama/wikipedia/gptneox-tok/en/',
    's2': 'oci://mosaicml-internal-datasets/s2orc/base/preconcat-gpt_neox/',
    'stack': 'oci://mosaicml-internal-datasets/stack-split-neox/'
}


In [ ]:
for domain in domains